In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as pl
import seaborn as sb
from matplotlib.pylab import rcParams
from matplotlib import rc
import os
import glob

In [2]:
# GRAPHICS SETUP:
rc('font', **{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text',usetex=False)
fontSize=16
sb.set_style('dark')
rcParams['ytick.labelsize'] = fontSize
rcParams['xtick.labelsize'] = fontSize
%matplotlib inline

In [ ]:
def GetDataDicts(fp):
    with nc.Dataset(fp) as ds:
        l3bVars = ds.groups['level-3_binned_data'].variables
        binlist = l3bVars['BinList'][:]
        wts = binlist['weights']
        bands = ['412', '443', '490', '510', '555', '670', '765', '865']
        rhoDict = {}
        rhoUncDict = {}
        rhotDict = {} 
        for band in bands:
            rhoDict[band] = np.pi * l3bVars['Rrs_%s' % band][:]['sum'] / wts
            rhoUncDict[band] = np.pi * l3bVars['Rrs_unc_%s' % band][:]['sum'] / wts
            rhotDict[band] = l3bVars['rhot_%s' % band][:]['sum'] / wts
    return rhoDict,rhoUncDict,rhotDict

In [ ]:
def Find95thPerctile(dataDict,ptile=.95,bands=['412','443','490',
                                          '510','555','670']):
    data95Dict = dict.fromkeys(bands)
    for band in bands:
        sortedData = np.sort(dataDict[band])
        data95Dict[band] = sortedData[int(np.floor(sortedData.size * ptile))]
    return rho95

def Get95Ptiles(biasList, dataDict):
    rho95Dict = dict.fromkeys(bands[:-2])
    for band in bands[:-2]:
        rho95Dict[band] = dict.fromkeys(biasList)
        for bias in biasList:
            data = dataDict[band][bias]
            sortedData = np.sort(data)
            rho95Dict[band][bias] = sortedData[int(np.floor(sortedData.size * .95))]
    return rho95Dict

In [ ]:
def PlotArtifacts(dataDict,artNoiseX,**kwargs):
    rng = kwargs.pop("rnge",(0,6e-3))
    colors = kwargs.pop("colors", ['black','indigo','blue','green'])
    savename = kwargs.pop("savename",None)
    band = kwargs.pop("band",'412')
    snr = kwargs.pop("snr", '500')
    f,ax = pl.subplots(figsize=(10,6))
    rhoThresh = 0.001
    f.suptitle(r'$\lambda=%s$ ABS. UNC. w/ NIR SNR=%s ' % (band,snr),fontsize=18)
    firstPass = True
    rho95Dict = dict.fromkeys(artNoiseX)
    for col,xsi in zip(colors,artNoiseX):
        lbl0 = float(xsi) * 100
        lbl = r'bias (%%$\rho_t$) : %.1f' % lbl0
        data = dataDict[band][xsi]
        sortedData = np.sort(data)
        rho95 = sortedData[int(np.floor(sortedData.size * .95))]
        pOK = data[data <= rhoThresh].size / data.size * 100
        lbl += r'$\rightarrow$ %.2f%% OK' % pOK
        if firstPass:
            ax.axvspan(0,rhoThresh,color='orange',alpha=0.2,label='threshold')
            firstPass = False
            ax.axvline(x=rho95,color=col,label="95%",linestyle='--')
        else:
            ax.axvline(x=rho95,color=col,linestyle='--')
        ax.hist(data,bins=100,range=rng,histtype='stepfilled',color=col,
                alpha=0.65,normed=True,label=lbl)

        #ax.set_title(r'$\sigma_{\rho_w}$ (art. unc: %s)' % xsi)
    ax.legend(loc='best',fontsize=fontSize-3)
    ax.set_xlabel(r'$ \rho_w$ inc. added bias as % $\rho_t$', fontsize=fontSize)
    ax.set_ylabel('Freq.',fontsize=fontSize)
    ax.ticklabel_format(axis='both',style='sci',scilimits=(1,3))
    if savename:
        f.savefig(savename,dpi=300,format='png')